# Blablattractivity - Local and tests -

## Data collection

We start with basic imports to use blablacar web api.

In [1]:
#%pip install pandas

In [2]:
#%pip install fsspec

In [3]:
#%pip install google-cloud-storage

In [4]:
#%pip install matplotlib

In [5]:
#%pip install seaborn

In [6]:
import json
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
import matplotlib.dates as mdates
import seaborn as sns
from google.cloud import storage
import datetime 
from math import ceil, floor

In [46]:
origin_destination_df = pd.read_csv("./origin_destination.csv", sep=";") # index_col = "Unnamed: 0"
origin_destination_df

,Origin,Destination,From_coordinate,To_coordinate,Distance
0,Paris,Brest,48.856614%7C2.352221,48.390394%7C-4.486076,"506,62"
1,Paris,Le Mans,48.856614%7C2.352221,48.00611%2C0.199556,"185,25"
2,Paris,Clermont-Ferrand,48.856614%7C2.352221,45.777221%2C3.087025,"346,83"
3,Paris,Dijon,48.856614%7C2.352221,47.32204%2C5.0414,"263,12"
4,Paris,Grenoble,48.856614%7C2.352221,45.18852%2C5.72452,"481,57"
...,...,...,...,...,...
337,Marseille,Nantes,43.296482%2C5.36978,47.218371%2C-1.553621,"696,17"
338,Marseille,Nice,43.296482%2C5.36978,43.710172%2C7.261953,"159,77"
339,Marseille,Toulouse,43.296482%2C5.36978,43.604652%2C1.444209,"319,55"
340,Marseille,Lyon,43.296482%2C5.36978,45.76404%2C4.83565,"277,47"


Convert the dataframe to a dictionnary :

In [47]:
route_dict = origin_destination_df.to_dict('index')

Dès qu'on reçoit l'API on part là-dessus : https://dev.blablacar.com/hc/en-us/articles/360012064820--Search-V3-API-Documentation

In [48]:
api_token = open("./api.txt","r").read()
api_url = "https://public-api.blablacar.com"

We have the basic parameters of our API call :

In [49]:
from_country = "FR" # string : ISO_3166_1
to_country = "FR" # string : ISO_3166_1
locale = "fr-FR" # string : locale in which the result will be displayed https://dev.blablacar.com/hc/en-us/articles/360008930020
currency = "EUR" # string : ISO_4217 currency
number_of_trips_we_want_returned = "100" # int : from 1 to 100, default is 10 
requested_seats = "1" # int : minimum number of seats requested, default value is 1
radius_in_meters = "6000" # int : radius of the circle around the coordinates in which trips will be searched from (in meters) - the default is 5000, so I chose 6000 because I noticed that 5000 is okay for cities but I want a margin of error for people meeting a bit away from cities

Then the date of the trip will be different everytime we run the code (it has to be next friday everytime), so we compute it here :

In [50]:
today = datetime.date.today()
friday = today + datetime.timedelta( (4-today.weekday()) % 7 )
next_friday_datetime = str(friday)+"T00:00:00"
start_date_local = next_friday_datetime

Create the dictionnary that is going to hold our information :

In [51]:
new_dict = {'trip_id':[],'price_of_the_trip':[],'date':[],'origin':[],'destination':[],'count_trips_on_this_route':[],'distance':[]}

Collect data : 

In [52]:
for index in route_dict:
    current_origin = route_dict[index]['Origin']
    current_destination = route_dict[index]['Destination']
    from_coordinate = route_dict[index]['From_coordinate'] # string : latitude, longitude - current_origin
    to_coordinate = route_dict[index]['To_coordinate'] # string : latitude, longitude - current_destination
    distance = route_dict[index]['Distance']
    
    blabla_query = api_url+"/api/v3/trips?key="+api_token+"&from_coordinate="+from_coordinate+"&from_country="+from_country+"&to_coordinate="+to_coordinate+"&to_country="+to_country+"&locale="+locale+"&currency="+currency+"&count="+number_of_trips_we_want_returned+"&start_date_local="+start_date_local+"&radius_in_meters="+radius_in_meters+"&sort=price:asc"
    response = requests.get(blabla_query)
    
    # print(response.status_code) # SHOULD BE 200
    
    json_trips = response.json() # it's a dictionnary
    
    try:
        print(json_trips)
        number_of_trips = json_trips["search_info"]["count"]
        for dict_trip in json_trips["trips"]:
            trip_id =  dict_trip["link"].split("id=")[1].split("-")[0]
            price_of_the_trip = dict_trip["price"]["amount"]
            new_dict['trip_id'].append(trip_id)
            new_dict['price_of_the_trip'].append(price_of_the_trip)
            new_dict['date'].append(start_date_local[:10])
            new_dict['origin'].append(current_origin)
            new_dict['destination'].append(current_destination)
            new_dict['count_trips_on_this_route'].append(number_of_trips)
            new_dict['distance'].append(distance)
    except:
        print("Unexpected error.")

{'link': 'https://www.blablacar.fr/search?fc=48.856614,2.352221&tc=48.390394,-4.486076&db=2020-08-28&de=2020-08-28', 'search_info': {'count': 0, 'full_trip_count': 0}, 'trips': []}
{'link': 'https://www.blablacar.fr/search?fc=48.856614,2.352221&tc=48.00611,0.199556&db=2020-08-28&de=2020-08-28', 'search_info': {'count': 8, 'full_trip_count': 1}, 'trips': [{'link': 'https://www.blablacar.fr/trip?source=CARPOOLING&id=2032935296-paris-saint-saturnin', 'waypoints': [{'date_time': '2020-08-28T12:40:00Z', 'place': {'city': 'Paris', 'address': 'Paris', 'latitude': 48.819054, 'longitude': 2.36014, 'country_code': 'FR'}}, {'date_time': '2020-08-28T14:40:00Z', 'place': {'city': 'Saint-Saturnin', 'address': 'Saint-Saturnin', 'latitude': 48.050178, 'longitude': 0.172327, 'country_code': 'FR'}}], 'price': {'amount': '10.00', 'currency': 'EUR'}, 'distance_in_meters': 195949, 'duration_in_seconds': 7200}, {'link': 'https://www.blablacar.fr/trip?source=CARPOOLING&id=2032229766-paris-saint-saturnin', 'w

{'link': 'https://www.blablacar.fr/search?fc=48.856614,2.352221&tc=45.18852,5.72452&db=2020-08-28&de=2020-08-28', 'search_info': {'count': 2, 'full_trip_count': 0}, 'trips': [{'link': 'https://www.blablacar.fr/trip?source=CARPOOLING&id=2032829701-paris-grenoble', 'waypoints': [{'date_time': '2020-08-28T10:00:00Z', 'place': {'city': 'Paris', 'address': "Porte d'Orléans", 'latitude': 48.823123, 'longitude': 2.325489, 'country_code': 'FR'}}, {'date_time': '2020-08-28T15:50:00Z', 'place': {'city': 'Grenoble', 'address': 'Grenoble Centre-ville, Grenoble', 'latitude': 45.18852, 'longitude': 5.72452, 'country_code': 'FR'}}], 'price': {'amount': '27.50', 'currency': 'EUR'}, 'distance_in_meters': 569031, 'duration_in_seconds': 21000}, {'link': 'https://www.blablacar.fr/trip?source=CARPOOLING&id=2032253336-paris-grenoble', 'waypoints': [{'date_time': '2020-08-28T13:00:00Z', 'place': {'city': 'Paris', 'address': '5 Passage Cottin, Paris', 'latitude': 48.887748, 'longitude': 2.346053, 'country_cod

{'link': 'https://www.blablacar.fr/search?fc=48.856614,2.352221&tc=44.837789,-0.57918&db=2020-08-28&de=2020-08-28', 'search_info': {'count': 7, 'full_trip_count': 2}, 'trips': [{'link': 'https://www.blablacar.fr/trip?source=CARPOOLING&id=2030527666-paris-bordeaux', 'waypoints': [{'date_time': '2020-08-28T13:10:00Z', 'place': {'city': 'Paris', 'address': "Porte d'Italie, Paris", 'latitude': 48.819054, 'longitude': 2.36014, 'country_code': 'FR'}}, {'date_time': '2020-08-28T19:10:00Z', 'place': {'city': 'Bordeaux', 'address': 'Rond Point de Rue d’Armagnac, Bordeaux', 'latitude': 44.822349, 'longitude': -0.554778, 'country_code': 'FR'}}], 'price': {'amount': '30.00', 'currency': 'EUR'}, 'vehicle': {'make': 'RENAULT', 'model': 'SCENIC'}, 'distance_in_meters': 580670, 'duration_in_seconds': 21600}, {'link': 'https://www.blablacar.fr/trip?source=CARPOOLING&id=2031595701-paris-bordeaux', 'waypoints': [{'date_time': '2020-08-28T09:00:00Z', 'place': {'city': 'Paris', 'address': "Gare de l'Est, P

{'link': 'https://www.blablacar.fr/search?fc=48.856614,2.352221&tc=43.710172,7.261953&db=2020-08-28&de=2020-08-28', 'search_info': {'count': 0, 'full_trip_count': 0}, 'trips': []}
{'link': 'https://www.blablacar.fr/search?fc=48.856614,2.352221&tc=43.604652,1.444209&db=2020-08-28&de=2020-08-28', 'search_info': {'count': 8, 'full_trip_count': 2}, 'trips': [{'link': 'https://www.blablacar.fr/trip?source=CARPOOLING&id=2032659681-paris-toulouse', 'waypoints': [{'date_time': '2020-08-28T07:00:00Z', 'place': {'city': 'Paris', 'address': 'Gare de Paris - Austerlitz, Paris', 'latitude': 48.841748, 'longitude': 2.366114, 'country_code': 'FR'}}, {'date_time': '2020-08-28T14:30:00Z', 'place': {'city': 'Toulouse', 'address': 'Matabiau, Toulouse', 'latitude': 43.610375, 'longitude': 1.44951, 'country_code': 'FR'}}], 'price': {'amount': '33.75', 'currency': 'EUR'}, 'vehicle': {'make': 'RENAULT', 'model': 'CLIO III'}, 'distance_in_meters': 677129, 'duration_in_seconds': 27000}, {'link': 'https://www.b

{'link': 'https://www.blablacar.fr/search?fc=48.856614,2.352221&tc=43.296482,5.36978&db=2020-08-28&de=2020-08-28', 'search_info': {'count': 1, 'full_trip_count': 0}, 'trips': [{'link': 'https://www.blablacar.fr/trip?source=CARPOOLING&id=2029583051-paris-marseille', 'waypoints': [{'date_time': '2020-08-28T22:00:00Z', 'place': {'city': 'Paris', 'address': 'Paris Gare de Lyon, Paris', 'latitude': 48.844303, 'longitude': 2.374377, 'country_code': 'FR'}}, {'date_time': '2020-08-29T06:10:00Z', 'place': {'city': 'Marseille', 'address': 'Saint-Charles, Marseille', 'latitude': 43.304005, 'longitude': 5.39009, 'country_code': 'FR'}}], 'price': {'amount': '57.50', 'currency': 'EUR'}, 'vehicle': {'make': 'VOLKSWAGEN', 'model': 'GOLF'}, 'distance_in_meters': 774840, 'duration_in_seconds': 29400}]}
{'link': 'https://www.blablacar.fr/search?fc=48.390394,-4.486076&tc=48.00611,0.199556&db=2020-08-28&de=2020-08-28', 'search_info': {'count': 1, 'full_trip_count': 0}, 'trips': [{'link': 'https://www.blabl

...

{'link': 'https://www.blablacar.fr/search?fc=49.258329,4.031696&tc=48.00611,0.199556&db=2020-08-28&de=2020-08-28', 'search_info': {'count': 2, 'full_trip_count': 0}, 'trips': [{'link': 'https://www.blablacar.fr/trip?source=CARPOOLING&id=2031714981-reims-le-mans', 'waypoints': [{'date_time': '2020-08-28T15:10:00Z', 'place': {'city': 'Reims', 'address': 'Reims - Gare Centre', 'latitude': 49.258589, 'longitude': 4.024975, 'country_code': 'FR'}}, {'date_time': '2020-08-28T18:50:00Z', 'place': {'city': 'Le Mans', 'address': 'D357, 72000 Le Mans, France', 'latitude': 48.0171461, 'longitude': 0.1425969, 'country_code': 'FR'}}], 'price': {'amount': '18.75', 'currency': 'EUR'}, 'distance_in_meters': 341959, 'duration_in_seconds': 13200}, {'link': 'https://www.blablacar.fr/trip?source=CARPOOLING&id=2023183591-reims-le-mans', 'waypoints': [{'date_time': '2020-08-28T16:20:00Z', 'place': {'city': 'Reims', 'address': '4 Imp. Léo Lagrange, 51100 Reims', 'latitude': 49.245115, 'longitude': 4.026451, '

{'link': 'https://www.blablacar.fr/search?fc=49.258329,4.031696&tc=48.573405,7.752111&db=2020-08-28&de=2020-08-28', 'search_info': {'count': 2, 'full_trip_count': 0}, 'trips': [{'link': 'https://www.blablacar.fr/trip?source=CARPOOLING&id=2026833906-reims-strasbourg', 'waypoints': [{'date_time': '2020-08-28T14:00:00Z', 'place': {'city': 'Reims', 'address': 'Gare de Reims, Reims', 'latitude': 49.258866, 'longitude': 4.024677, 'country_code': 'FR'}}, {'date_time': '2020-08-28T17:30:00Z', 'place': {'city': 'Strasbourg', 'address': 'Gare de Strasbourg, Strasbourg', 'latitude': 48.585232, 'longitude': 7.735018, 'country_code': 'FR'}}], 'price': {'amount': '21.25', 'currency': 'EUR'}, 'vehicle': {'make': 'CITROEN', 'model': 'C3'}, 'distance_in_meters': 353102, 'duration_in_seconds': 12600}, {'link': 'https://www.blablacar.fr/trip?source=CARPOOLING&id=2031867776-reims-strasbourg', 'waypoints': [{'date_time': '2020-08-28T17:10:00Z', 'place': {'city': 'Reims', 'address': 'Reims - Gare Centre', '

{'link': 'https://www.blablacar.fr/search?fc=48.117266,-1.677793&tc=48.390394,-4.486076&db=2020-08-28&de=2020-08-28', 'search_info': {'count': 3, 'full_trip_count': 1}, 'trips': [{'link': 'https://www.blablacar.fr/trip?source=CARPOOLING&id=2028360731-rennes-brest', 'waypoints': [{'date_time': '2020-08-28T15:30:00Z', 'place': {'city': 'Rennes', 'address': 'Villejean-Université, Rennes', 'latitude': 48.121265, 'longitude': -1.704042, 'country_code': 'FR'}}, {'date_time': '2020-08-28T18:30:00Z', 'place': {'city': 'Brest', 'address': 'Gare de Brest, Brest', 'latitude': 48.387563, 'longitude': -4.480924, 'country_code': 'FR'}}], 'price': {'amount': '11.25', 'currency': 'EUR'}, 'vehicle': {'make': 'VOLKSWAGEN', 'model': 'PASSAT SW'}, 'distance_in_meters': 241076, 'duration_in_seconds': 10800}, {'link': 'https://www.blablacar.fr/trip?source=CARPOOLING&id=2032548616-rennes-brest', 'waypoints': [{'date_time': '2020-08-28T19:50:00Z', 'place': {'city': 'Rennes', 'address': 'Rennes', 'latitude': 4

{'link': 'https://www.blablacar.fr/search?fc=48.117266,-1.677793&tc=45.777221,3.087025&db=2020-08-28&de=2020-08-28', 'search_info': {'count': 1, 'full_trip_count': 0}, 'trips': [{'link': 'https://www.blablacar.fr/trip?source=CARPOOLING&id=2033935751-rennes-clermont-ferrand', 'waypoints': [{'date_time': '2020-08-28T10:20:00Z', 'place': {'city': 'Rennes', 'address': 'Métro La Poterie, Rennes', 'latitude': 48.087394, 'longitude': -1.644323, 'country_code': 'FR'}}, {'date_time': '2020-08-28T16:30:00Z', 'place': {'city': 'Clermont-Ferrand', 'address': 'Gare de Clermont La Pardieu, Clermont-Ferrand', 'latitude': 45.766889, 'longitude': 3.134429, 'country_code': 'FR'}}], 'price': {'amount': '37.50', 'currency': 'EUR'}, 'vehicle': {'make': 'VOLKSWAGEN', 'model': 'POLO 4'}, 'distance_in_meters': 580126, 'duration_in_seconds': 22200}]}
{'link': 'https://www.blablacar.fr/search?fc=48.117266,-1.677793&tc=47.32204,5.0414&db=2020-08-28&de=2020-08-28', 'search_info': {'count': 0, 'full_trip_count': 0

{'message': 'API rate limit exceeded'}
Unexpected error.
{'message': 'API rate limit exceeded'}
Unexpected error.
{'message': 'API rate limit exceeded'}
Unexpected error.
{'message': 'API rate limit exceeded'}
Unexpected error.
{'message': 'API rate limit exceeded'}
Unexpected error.
{'message': 'API rate limit exceeded'}
Unexpected error.
{'message': 'API rate limit exceeded'}
Unexpected error.
{'message': 'API rate limit exceeded'}
Unexpected error.
{'message': 'API rate limit exceeded'}
Unexpected error.
{'message': 'API rate limit exceeded'}
Unexpected error.
{'message': 'API rate limit exceeded'}
Unexpected error.
{'message': 'API rate limit exceeded'}
Unexpected error.
{'message': 'API rate limit exceeded'}
Unexpected error.
{'message': 'API rate limit exceeded'}
Unexpected error.
{'message': 'API rate limit exceeded'}
Unexpected error.
{'message': 'API rate limit exceeded'}
Unexpected error.
{'message': 'API rate limit exceeded'}
Unexpected error.
{'message': 'API rate limit exc

In [26]:
#new_dict

{'trip_id': ['2029583051'],
 'price_of_the_trip': ['57.50'],
 'date': ['2020-08-28'],
 'origin': ['Paris'],
 'destination': ['Marseille'],
 'count_trips_on_this_route': [1],
 'distance': [660.4954812373862]}

Feed it to the dataframe :

In [53]:
new_df = pd.DataFrame.from_dict(new_dict)
new_df.price_of_the_trip = new_df.price_of_the_trip.astype('float')    

Save the df in folder here :

In [54]:
#old_df = pd.read_csv("./blablacar_rennes_brest.csv",index_col = "Unnamed: 0") 
#old_df.head()
#df_trips = old_df.append(new_df, ignore_index=True, sort=False)
#df_trips.head()
new_df.to_csv("./blablacar_db.csv")